# Time comparison

In this notebook we will analyze different size datasets (known datasets) to see how long does triku take to analyze the datasets. The time comparison will be done with 3 settings:
* All features (after basic removal), 1 processor
* Filtered features (after removal, to get ~10000 features), 1 processor
* Filtered features, several processors (2, 4, 8, 16)

We will compare these times with the time required for the rest of processing times of a basic preprocessing pipeline (get neighbors, PCA, UMAP, leiden, etc.) to see how long does running triku take in comparison with the rest of the steps of the pipeline.

In [ ]:
!pip uninstall triku -y
!cd ../triku && python setup.py install

In [1]:
%matplotlib inline

In [2]:
# Imports here
import scanpy as sc
import scanpy.external as sce
import harmonypy as hpy

import numpy as np
import scipy.stats as sts
import scipy.optimize as opt
import scipy.signal as sgn
import pandas as pd

import triku as tk

import logging
import os
from tqdm.notebook import tqdm

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# Especial imports
from sklearn.decomposition import PCA
from umap.umap_ import fuzzy_simplicial_set, nearest_neighbors

# Parallel imports
import ray
from numba import jit, njit

random_state = 10

import time
import mygene

In [3]:
ray.logger.setLevel(logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [4]:
from triku_nb_code.palettes_and_cmaps import magma, bold_and_vivid

In [5]:
def read_adata(dataset_file, backup_url):
    try:
        adata = sc.read(dataset_file, backup_url=backup_url)
    except:
        try:
            adata = sc.read_10x_h5(dataset_file, backup_url=backup_url)
        except:
            adata = sc.read_10x_mtx(dataset_file)
    
    return adata

def processing_pipeline(dataset_file, n_genes_by_counts=4000, n_neighbors=None,
                        min_pct_counts_mt=0, pct_counts_mt=25, min_genes=50, min_cells=10, transpose=False, backup_url='', 
                        get_int=True, plot_graphs=True, prefilter=True):
    t_dict = {}
    
    # Dataset loading and basic profiling
    if isinstance(dataset_file, str):
        adata = read_adata(dataset_file, backup_url).copy()
        if transpose:
            adata = adata.transpose()
            
    elif isinstance(dataset_file, list):
        list_adatas = [read_adata(os.getcwd() + '/data/' + i.split('/')[-1], backup_url=i) for i in dataset_file]
        if transpose:
            list_adatas = [i.transpose() for i in list_adatas]
        adata = sc.AnnData.concatenate(*list_adatas)
    
    elif isinstance(dataset_file, sc.AnnData):
        adata = dataset_file.copy()
    
    # If genes are ENSMUG or like that, we set a query to rename the names
    if np.any(['ENSMU' in i for i in adata.var_names]) | np.any(['ENSG' in i for i in adata.var_names]):
        mg = mygene.MyGeneInfo()
        query = mg.querymany(adata.var_names, scopes='ensembl.gene', returnall=True)
        genes = [i['symbol'] if 'symbol' in i else '' for i in query['out']]
        adata.var_names = genes
        
    adata.var_names_make_unique() 

    t0 = time.time()
    sc.pp.filter_cells(adata, min_genes=min_genes)
    sc.pp.filter_genes(adata, min_cells=min_cells)
    tf = time.time()
    print('gene/cell filter', tf-t0)
    t_dict['gene/cell filter'] = tf-t0
    
    if prefilter:
        # Dataset profiling - get the graphs to set the n_genes_by_counts and pct_counts_mt
        t0 = time.time()
        adata.var['mt'] = adata.var_names.str.startswith('MT-') | adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
        sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

        if plot_graphs:
            sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
                     jitter=0.4, multi_panel=True)
            sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
            sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

        tf = time.time()
        print('QC_metrics', tf-t0)
        t_dict['QC_metrics'] = tf-t0

        t0 = time.time()
        adata = adata[adata.obs.n_genes_by_counts < n_genes_by_counts, :]
        adata = adata[adata.obs.pct_counts_mt < pct_counts_mt, :]
        adata = adata[min_pct_counts_mt < adata.obs.pct_counts_mt, :]
        sc.pp.filter_cells(adata, min_genes=1)
        sc.pp.filter_genes(adata, min_counts=1)
        tf = time.time()
        print('filter_cells_genes', tf-t0)
        t_dict['filter_cells_genes'] = tf-t0

        if not get_int:
            # Normalize and log-transforming
            t0 = time.time()
            sc.pp.normalize_total(adata)
            tf = time.time()
            print('normalize', tf-t0)
            t_dict['normalize'] = tf-t0

            t0 = time.time()
            sc.pp.log1p(adata)
            tf = time.time()
            print('log1p', tf-t0)
            t_dict['log1p'] = tf-t0

    print(adata)

    t0 = time.time()
    sc.pp.pca(adata)
    tf = time.time()
    print('pca', tf-t0)
    t_dict['pca'] = tf-t0
    
    if n_neighbors is None:
        n_neighbors=int(0.5 * len(adata) ** 0.5)
    
    # Preparation for FS (PCA, kNN, etc.)
    if 'batch' in adata.obs:
        print('batch in adata')
        t0 = time.time()
        sce.pp.bbknn(adata)
        tf = time.time()
        print('bbknn', tf-t0)
        t_dict['bbknn'] = tf-t0
        
            
        t0 = time.time()
        sce.pp.harmony_integrate(adata, key='batch')
        tf = time.time()
        print('harmony', tf-t0)
        t_dict['harmony'] = tf-t0
        
        t0 = time.time()
        sc.pp.neighbors(adata, n_neighbors=n_neighbors, metric='cosine', use_rep='X_pca_harmony')
        tf = time.time()
        print('knn', tf-t0)
        t_dict['knn'] = tf-t0
    else:
        print('batch NOT in adata')
        t0 = time.time()
        sc.pp.neighbors(adata, n_neighbors=n_neighbors, metric='cosine')
        tf = time.time()
        print('knn', tf-t0)
        t_dict['knn'] = tf-t0

    # FS
    sc.pp.filter_genes(adata, min_cells=5)
    
    t0 = time.time()
    tk.tl.triku(adata)
    tf = time.time()
    print('triku', tf-t0)
    t_dict['triku'] = tf-t0

    

    
    # Dimensionality reduction
    t0 = time.time()
    sc.tl.tsne(adata, n_jobs=16)
    tf = time.time()
    print('t-SNE', tf-t0)
    t_dict['t-SNE'] = tf-t0

    t0 = time.time()
    sc.tl.umap(adata)
    tf = time.time()
    print('UMAP', tf-t0)
    t_dict['UMAP'] = tf-t0
    
    # Clustering
    t0 = time.time()
    sc.tl.leiden(adata)
    tf = time.time()
    print('leiden', tf-t0)
    t_dict['leiden'] = tf-t0
    
    t0 = time.time()
    sc.tl.louvain(adata)
    tf = time.time()
    print('louvain', tf-t0)
    t_dict['louvain'] = tf-t0
    
    # PAGA / trajectory analysis
    t0 = time.time()
    sc.tl.paga(adata)
    tf = time.time()
    print('PAGA', tf-t0)
    t_dict['PAGA'] = tf-t0
    
    return t_dict

# PBMC datasets (1, 5, 68k)

## 1k

In [6]:
t_dict_pbmc_1k = processing_pipeline(dataset_file=os.getcwd() + '/data/10x/pbmc_1k_v3_filtered_feature_bc_matrix.h5', n_genes_by_counts=3000, pct_counts_mt=20, plot_graphs=False,
                                    backup_url='https://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_v3/pbmc_1k_v3_filtered_feature_bc_matrix.h5')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.


gene/cell filter 0.1764531135559082
QC_metrics 0.06603288650512695
filter_cells_genes 0.06322169303894043
AnnData object with n_obs × n_vars = 901 × 12564
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_counts'
pca 2.3754818439483643
batch NOT in adata
knn 4.574864864349365
triku 5.610928535461426
t-SNE 2.304821014404297


... storing 'feature_types' as categorical
... storing 'genome' as categorical


UMAP 2.431525945663452
leiden 0.06370878219604492
louvain 0.04371166229248047
PAGA 0.1453261375427246


In [7]:
t_dict_pbmc_1k

{'gene/cell filter': 0.1764531135559082,
 'QC_metrics': 0.06603288650512695,
 'filter_cells_genes': 0.06322169303894043,
 'pca': 2.3754818439483643,
 'knn': 4.574864864349365,
 'triku': 5.610928535461426,
 't-SNE': 2.304821014404297,
 'UMAP': 2.431525945663452,
 'leiden': 0.06370878219604492,
 'louvain': 0.04371166229248047,
 'PAGA': 0.1453261375427246}

## 5k

In [8]:
t_dict_pbmc_5k = processing_pipeline(dataset_file=os.getcwd() + '/data/10x/5k_pbmc_protein_v3_filtered_feature_bc_matrix.h5', n_genes_by_counts=4000, pct_counts_mt=20, plot_graphs=False,
                                    backup_url='https://cf.10xgenomics.com/samples/cell-exp/3.0.2/5k_pbmc_protein_v3/5k_pbmc_protein_v3_filtered_feature_bc_matrix.h5')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


gene/cell filter 0.3525125980377197


Trying to set attribute `.obs` of view, copying.


QC_metrics 0.2083292007446289
filter_cells_genes 0.28251194953918457
AnnData object with n_obs × n_vars = 4094 × 15000
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_counts'
pca 12.655788898468018
batch NOT in adata
knn 0.9816465377807617
triku 13.484415769577026
t-SNE 11.23947286605835
UMAP 10.400574445724487
leiden 0.758129358291626


... storing 'feature_types' as categorical
... storing 'genome' as categorical


louvain 0.5756673812866211
PAGA 0.3716881275177002


In [9]:
t_dict_pbmc_5k

{'gene/cell filter': 0.3525125980377197,
 'QC_metrics': 0.2083292007446289,
 'filter_cells_genes': 0.28251194953918457,
 'pca': 12.655788898468018,
 'knn': 0.9816465377807617,
 'triku': 13.484415769577026,
 't-SNE': 11.23947286605835,
 'UMAP': 10.400574445724487,
 'leiden': 0.758129358291626,
 'louvain': 0.5756673812866211,
 'PAGA': 0.3716881275177002}

## 68k

In [ ]:
!cd {os.getcwd()}/data/10x && wget https://cf.10xgenomics.com/samples/cell-exp/1.1.0/fresh_68k_pbmc_donor_a/fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz

In [ ]:
!cd {os.getcwd()}/data/10x && tar zxvf fresh_68k_pbmc_donor_a_filtered_gene_bc_matrices.tar.gz

In [10]:
t_dict_pbmc_68k = processing_pipeline(dataset_file=os.getcwd() + '/data/10x/filtered_matrices_mex/hg19', n_genes_by_counts=1000, pct_counts_mt=5, plot_graphs=False,
                                     backup_url='', n_neighbors=30)

gene/cell filter 1.3913171291351318
QC_metrics 0.8236515522003174


Trying to set attribute `.obs` of view, copying.


filter_cells_genes 1.2771224975585938
AnnData object with n_obs × n_vars = 66758 × 15331
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_counts'
pca 27.000394344329834
batch NOT in adata
knn 31.976906538009644
triku 31.641143798828125
t-SNE 260.266250371933
UMAP 53.95405077934265
leiden 28.115334510803223
louvain 26.543845653533936
PAGA 2.922435998916626


In [11]:
t_dict_pbmc_68k

{'gene/cell filter': 1.3913171291351318,
 'QC_metrics': 0.8236515522003174,
 'filter_cells_genes': 1.2771224975585938,
 'pca': 27.000394344329834,
 'knn': 31.976906538009644,
 'triku': 31.641143798828125,
 't-SNE': 260.266250371933,
 'UMAP': 53.95405077934265,
 'leiden': 28.115334510803223,
 'louvain': 26.543845653533936,
 'PAGA': 2.922435998916626}

# Subramanian et al. - Kidney organoid (300k, batches)

In [12]:
list_GSM = ['GSM4044536', 'GSM4044537', 'GSM4044538', 'GSM4044539', 'GSM4044540', 'GSM4044541', 'GSM4044542', 'GSM4044543', 'GSM4044544']

In [13]:
kidney_org_dir = os.getcwd() + '/data/kidney'
os.makedirs(kidney_org_dir, exist_ok=True)

In [ ]:
# If it doesn't work, download it manually
!wget https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE136314&format=file -O {kidney_org_dir}/GSE136314_RAW.tar

In [ ]:
!tar -xvf {kidney_org_dir}/GSE136314_RAW.tar -C {kidney_org_dir}

In [ ]:
for GSM in list_GSM:
    files = [i for i in os.listdir(kidney_org_dir) if GSM in i]
    os.makedirs(f'{kidney_org_dir}/{GSM}', exist_ok=True)
    mtx, barcodes, genes = [i for i in files if 'gene_count.mtx' in i ][0], [i for i in files if 'barcodes.tsv.gz' in i ][0], [i for i in files if 'genes.tsv.gz' in i ][0]
    os.rename(f'{kidney_org_dir}/{mtx}', f'{kidney_org_dir}/{GSM}/matrix.mtx.gz')
    os.rename(f'{kidney_org_dir}/{barcodes}', f'{kidney_org_dir}/{GSM}/barcodes.tsv.gz')
    os.rename(f'{kidney_org_dir}/{genes}', f'{kidney_org_dir}/{GSM}/features.tsv.gz')

In [ ]:
list_adatas = []
for GSM in list_GSM:
    print(GSM)
    adata = sc.read_mtx(f'{kidney_org_dir}/{GSM}/matrix.mtx.gz').transpose()
    adata.var_names = pd.read_csv(f'{kidney_org_dir}/{GSM}/features.tsv.gz', header=None)[0]
    adata.obs_names = pd.read_csv(f'{kidney_org_dir}/{GSM}/barcodes.tsv.gz', header=None)[0]
    list_adatas.append(adata)
    
adata_kidney = sc.AnnData.concatenate(*list_adatas, join='outer')
adata_kidney.write_h5ad(f'{kidney_org_dir}/adata_kidney.h5ad')

In [14]:
adata_kidney = sc.read(f'{kidney_org_dir}/adata_kidney.h5ad')

In [15]:
t_dict_kidney = processing_pipeline(adata_kidney, prefilter=True, min_cells=100, pct_counts_mt=5, n_genes_by_counts=3500, n_neighbors=30, plot_graphs=False,)

gene/cell filter 11.65590524673462
QC_metrics 6.921982049942017


Trying to set attribute `.obs` of view, copying.


filter_cells_genes 11.225468158721924
AnnData object with n_obs × n_vars = 242106 × 19067
    obs: 'batch', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_counts'
pca 167.6449258327484
batch in adata
bbknn 82.71136426925659


2021-08-31 16:23:05,999 - harmonypy - INFO - Iteration 1 of 10
2021-08-31 16:26:02,736 - harmonypy - INFO - Iteration 2 of 10
2021-08-31 16:29:00,915 - harmonypy - INFO - Iteration 3 of 10
2021-08-31 16:32:00,430 - harmonypy - INFO - Converged after 3 iterations


harmony 639.8956282138824
knn 58.98443937301636
triku 315.4680349826813
t-SNE 928.0861029624939
UMAP 221.78708028793335
leiden 223.75757813453674
louvain 119.32232880592346
PAGA 15.196204423904419


# T cells (60k, batches)

In [18]:
tcell_dir = os.getcwd() + '/data/tcell'
os.makedirs(tcell_dir, exist_ok=True)

In [ ]:
# If it doesn't work, download it manually
!wget https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE150132 -O {tcell_dir}/GSE150132_RAW.tar

In [ ]:
!tar -xvf {tcell_dir}/GSE150132_RAW.tar -C {tcell_dir}

In [ ]:
list_GSM = ['GSM4524029', 'GSM4524030', 'GSM4524031', 'GSM4524032', 'GSM4524033']
for GSM in list_GSM:
    files = [i for i in os.listdir(tcell_dir) if GSM in i]
    os.makedirs(f'{tcell_dir}/{GSM}', exist_ok=True)
    mtx, barcodes, genes = [i for i in files if 'matrix.mtx' in i ][0], [i for i in files if 'barcodes.tsv.gz' in i ][0], [i for i in files if 'features.tsv.gz' in i ][0]
    os.rename(f'{tcell_dir}/{mtx}', f'{tcell_dir}/{GSM}/matrix.mtx.gz')
    os.rename(f'{tcell_dir}/{barcodes}', f'{tcell_dir}/{GSM}/barcodes.tsv.gz')
    os.rename(f'{tcell_dir}/{genes}', f'{tcell_dir}/{GSM}/features.tsv.gz')

In [ ]:
list_adatas = []
for GSM in list_GSM:
    print(GSM)
    adata = sc.read_mtx(f'{tcell_dir}/{GSM}/matrix.mtx.gz').transpose()
    adata.var_names = pd.read_csv(f'{tcell_dir}/{GSM}/features.tsv.gz', header=None)[0]
    adata.obs_names = pd.read_csv(f'{tcell_dir}/{GSM}/barcodes.tsv.gz', header=None)[0]
    list_adatas.append(adata)
    
adata_tcell = sc.AnnData.concatenate(*list_adatas, join='outer')
adata_tcell.var_names = [i.split('\t')[1] for i in adata_tcell.var_names]
adata_tcell.write_h5ad(f'{tcell_dir}/adata_tcell.h5ad')

In [19]:
adata_tcell = sc.read(f'{tcell_dir}/adata_tcell.h5ad')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [20]:
t_dict_tcell =  processing_pipeline(adata_tcell, n_genes_by_counts=2000, min_pct_counts_mt=0, pct_counts_mt=8, 
                                       min_genes=50, min_cells=10, transpose=False, get_int=True, plot_graphs=False, prefilter=True, n_neighbors=30,
                                       backup_url='')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


gene/cell filter 4.304690599441528
QC_metrics 2.52275013923645


Trying to set attribute `.obs` of view, copying.


filter_cells_genes 3.5770392417907715
AnnData object with n_obs × n_vars = 64132 × 18201
    obs: 'batch', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_counts'
pca 53.858011960983276
batch in adata
bbknn 16.25207757949829


2021-09-01 17:23:18,930 - harmonypy - INFO - Iteration 1 of 10
2021-09-01 17:24:29,645 - harmonypy - INFO - Iteration 2 of 10
2021-09-01 17:25:30,638 - harmonypy - INFO - Iteration 3 of 10
2021-09-01 17:26:25,982 - harmonypy - INFO - Iteration 4 of 10
2021-09-01 17:27:20,766 - harmonypy - INFO - Iteration 5 of 10
2021-09-01 17:28:16,228 - harmonypy - INFO - Iteration 6 of 10
2021-09-01 17:28:54,960 - harmonypy - INFO - Iteration 7 of 10
2021-09-01 17:29:29,717 - harmonypy - INFO - Iteration 8 of 10
2021-09-01 17:30:02,409 - harmonypy - INFO - Converged after 8 iterations


harmony 450.6034576892853
knn 17.63447904586792
triku 61.609858751297
t-SNE 317.4437565803528
UMAP 54.64051556587219
leiden 35.97402501106262
louvain 28.63161873817444
PAGA 4.470667362213135
